In [1]:
import tensorflow as tf
import pandas as pd
import os

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
train = pd.read_csv('data/iris_training.csv', names=CSV_COLUMN_NAMES, header=0)
train_x, train_y = train, train.pop('Species')
train_x = train_x.to_dict(orient='list')
train_y = train_y.values

val = pd.read_csv('data/iris_test.csv', names=CSV_COLUMN_NAMES, header=0)
val_x, val_y = val, val.pop('Species')
val_x = val_x.to_dict(orient='list')
val_y = val_y.values

In [3]:
print(train_x.keys())

FEATURE_COLUMNS = [
    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

dict_keys(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'])


In [4]:
N_CLASSES = 3
N_INPUT_FEATURES = 4
BATCH_SIZE = 32
STEPS=1000
MODEL_DIR = 'model'
EXPORT_DIR = os.path.join(MODEL_DIR, 'export', 'Servo')

# Input functions

In [5]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    # Batch the examples.
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

def predict_input_fn(features, batch_size):
    """An input function for prediction"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(features)

    # Batch the examples.
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

def serving_input_fn():
    """
    An input funtion for serving
    
    Returns a `serving_input_receiver_fn` that 
    takes no argument and 
    returns a `tf.estimator.export.ServingInputReceiver` or `tf.estimator.export.TensorServingInputReceiver`.
    """
    features = {
        'SepalLength': tf.placeholder(shape=[None], dtype=tf.float32), 
        'SepalWidth': tf.placeholder(shape=[None], dtype=tf.float32),
        'PetalLength': tf.placeholder(shape=[None], dtype=tf.float32), 
        'PetalWidth': tf.placeholder(shape=[None], dtype=tf.float32)
    }
    return tf.estimator.export.build_raw_serving_input_receiver_fn(features)

In [6]:
# test train_input_fn
# -------------------
dataset = train_input_fn(train_x, train_y, batch_size=5)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    f, l = sess.run(next_element)
    print(f)
    print(l)
# -------------------

W0720 20:26:11.013882 17744 deprecation.py:323] From <ipython-input-6-fcd552b503b1>:4: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'SepalLength': array([6.5, 4.4, 7.6, 4.9, 6.3], dtype=float32), 'SepalWidth': array([3. , 3.2, 3. , 3.1, 2.3], dtype=float32), 'PetalLength': array([5.8, 1.3, 6.6, 1.5, 4.4], dtype=float32), 'PetalWidth': array([2.2, 0.2, 2.1, 0.1, 1.3], dtype=float32)}
[2 0 2 0 1]


# Model function

The model function has the following call signature:

`def model_fn(
    features, # This is batch_features from input_fn
    labels,   # This is batch_labels from input_fn
    mode,     # An instance of tf.estimator.ModeKeys
    params):  # Additional configuration, passed from tf.estimator.Estimator`
    
and returns an instance of `tf.estimator.EstimatorSpec`.

In [7]:
def create_model(n_classes):
    inputs = tf.keras.layers.Input(shape=(N_INPUT_FEATURES,)) # (None, N_INPUT_FEATURES)
    x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(inputs) # (None, 10)
    x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x) # (None, 10)
    x = tf.keras.layers.Dense(n_classes)(x) # (None, 10)
    model = tf.keras.Model(inputs, x)
    return model

In [8]:
# test create_model
# -----------------
model = create_model(3)

dataset = train_input_fn(train_x, train_y, batch_size=5)
iterator = dataset.make_one_shot_iterator()
next_element, _ = iterator.get_next()

inputs = tf.keras.layers.DenseFeatures(FEATURE_COLUMNS)(next_element) # (None, N_INPUT_FEATURES)
logits = model(inputs) # (None, 3)

var_init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(var_init)
    print(logits.eval())
# -----------------

W0720 20:26:11.360229 17744 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[[-0.5931164   0.28143415 -0.3913166 ]
 [-0.47901395  0.23427066 -0.39212173]
 [-0.48497716  0.27129757 -0.51716274]
 [-0.4879142   0.26547086 -0.46955964]
 [-2.8710532   0.57653165  1.0698833 ]]


Depending on the value of `mode`, different arguments for `tf.estimator.EstimatorSpec` are required. Namely
* For `mode == ModeKeys.TRAIN`: required fields are `loss` and `train_op`.
* For `mode == ModeKeys.EVAL`: required field is `loss`.
* For `mode == ModeKeys.PREDICT`: required fields are `predictions`.

In [9]:
def model_fn(features, labels, mode, params):
    '''
    inputs
        features: This is batch_features (first output) from input_fn 
        labels: This is batch_labels (second output) from input_fn, shape (None,)
        mode: An instance of tf.estimator.ModeKeys
        params: Additional configuration, passed from tf.estimator.Estimator
                For this example we only pass `n_classes`.
    returns:
        an instance of tf.estimator.EstimatorSpec
    '''
    model = create_model(params['n_classes'])
    
    inputs = tf.keras.layers.DenseFeatures(FEATURE_COLUMNS)(features) # shape (None, N_INPUT_FEATURES)
    
    if mode == tf.estimator.ModeKeys.PREDICT: # only invoked when call tf.estimator.Estimator.predict
        logits = model(inputs, 
                       training=False) # this param is used when there are layers (e.g. Dropout layers)
                                       # that behave differently in training and inference
        predictions = {
            'classes': tf.math.argmax(logits, axis=1), # shape (None, )
            'probabilities': tf.nn.softmax(logits) # shape (None, n_classes)
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.EVAL: # only invoked when call tf.estimator.Estimator.evaluate
        logits = model(inputs, 
                       training=False) # this param is used when there are layers (e.g. Dropout layers)
                                       # that behave differently in training and inference
        # Compute loss
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, # labels shape (None, )
                                                      logits=logits) # logits shape (None, n_classes)  
        # Compute predictions
        predicted_classes = tf.math.argmax(logits, axis=1) # shape (None, )
        # Compute evaluation metrics
        accuracy = tf.metrics.accuracy(labels=labels,
                                       predictions=predicted_classes,
                                       name='eval_accuracy')
        
        return tf.estimator.EstimatorSpec(mode, 
                                          loss=loss, 
                                          eval_metric_ops={'eval_accuracy': accuracy}) # record to Tensorboard
    
    if mode == tf.estimator.ModeKeys.TRAIN: # only invoked when call tf.estimator.Estimator.train
        logits = model(inputs, 
                       training=True) # this param is used when there are layers (e.g. Dropout layers)
                                       # that behave differently in training and inference
        # Compute loss
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, # labels shape (None, )
                                                      logits=logits) # logits shape (None, n_classes)
        
        # Compute predictions
        predicted_classes = tf.math.argmax(logits, axis=1) # shape (None, )
        # Compute metrics
        acc, acc_op = tf.metrics.accuracy(labels=labels,
                                          predictions=predicted_classes,
                                          name='train_accuracy')
        # Record train accuracy to Tensorboard
        tf.summary.scalar('train_accuracy', acc_op)
        
        # Create an optimizer and training op
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss, 
                                      global_step=tf.train.get_global_step()) # it's important to pass global_step
                                                                              # to minimize() method in order to record
                                                                              # the x-coordinate for Tensorboard graphs
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

# The custom Estimator

## Create a custom Estimator:

In [10]:
classifier = tf.estimator.Estimator(model_fn=model_fn,
                                    model_dir=MODEL_DIR,
                                    params={'n_classes': N_CLASSES}) # pass to model_fn params

## Train the model:

In [11]:
classifier.train(input_fn=lambda: train_input_fn(train_x, train_y, batch_size=BATCH_SIZE), 
                 steps=STEPS)

W0720 20:26:12.182610 17744 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0720 20:26:12.304316 17744 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0720 20:26:12.378116 17744 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\training\adagrad.py:76: calling Constant.__init__ (from tensorflow.python.

Here we wrap up our `input_fn` call in a `lambda` to capture the arguments while providing an input function that takes no arguments, as expected by the `Estimator`.

## Evaluate the model:

In [12]:
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(val_x, val_y, batch_size=BATCH_SIZE))

W0720 20:26:14.911648 17744 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [13]:
eval_result

{'eval_accuracy': 0.96666664, 'loss': 0.05541057, 'global_step': 1000}

## Export the model

In [14]:
classifier.export_saved_model(export_dir_base=EXPORT_DIR, serving_input_receiver_fn=serving_input_fn())

W0720 20:26:15.536814 17744 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


b'model\\export\\1563629175'

## Generate predictions from the model

In [15]:
class_name = ['Setosa', 'Versicolor' , 'Virginica']
test_y = ['Setosa', 'Versicolor', 'Virginica']
test_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [16]:
predictions = classifier.predict(input_fn=lambda: predict_input_fn(test_x, batch_size=BATCH_SIZE))

In [17]:
for i, (true, pred) in enumerate(zip(test_y, predictions)):
    print('SAMPLE {}:'.format(i+1))
    print('Prediction: ', class_name[pred['classes']])
    print('True: ', test_y[i])
    for c in range(N_CLASSES):
        print('Class `{}` confidence: {}'.format(class_name[c], pred['probabilities'][c]))
    
    print('\n##################################################')
    print('##################################################\n')

SAMPLE 1:
Prediction:  Setosa
True:  Setosa
Class `Setosa` confidence: 0.9991546869277954
Class `Versicolor` confidence: 0.000845316331833601
Class `Virginica` confidence: 1.3263680154196322e-09

##################################################
##################################################

SAMPLE 2:
Prediction:  Versicolor
True:  Versicolor
Class `Setosa` confidence: 0.00015223266382236034
Class `Versicolor` confidence: 0.9976334571838379
Class `Virginica` confidence: 0.0022142541129142046

##################################################
##################################################

SAMPLE 3:
Prediction:  Virginica
True:  Virginica
Class `Setosa` confidence: 3.49274927202714e-07
Class `Versicolor` confidence: 0.10266430675983429
Class `Virginica` confidence: 0.897335410118103

##################################################
##################################################

